In [2]:
!pip install cairosvg

     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.8MB/s eta 0:00:01
  Stored in directory: /Users/mallikapriyakhullar/Library/Caches/pip/wheels/40/76/48/f1effadceea83b32e7d957dd0f92db4db8b537d7b72b4ef374
Successfully built cairocffi


In [4]:
import os
import numpy as np
import subprocess
from fingerprintProcessing import getSMILEsFromFileWithHeader, getActInactListForATarget, constructDataMatricesForATarget, constructDataMatricesForATargetOtherClassifier, getTopNModels
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from random import shuffle
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.metrics import confusion_matrix
import sys


IMG_SIZE = 200


training_dataset_path = "/Users/trman/OneDrive/Projects/DrugDiscovery/TrainingDatasets"
images_path = "../images200"
TEMP_IMG_OUTPUT_PATH = "../tempImage"

import sys

def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

def constructECFP4FeaturesVectors(target_id):
    train_test_data = []
    prob_count = 0
    count = 0
    compound_smiles_dict = getSMILEsFromFileWithHeader("chembl_23_chemreps.txt")
    act_list, inact_list = getActInactListForATarget(target_id, "act_inact_comps_10.0_20.0_chembl_preprocessed_sp_b_pchembl_data_blast_comp_20.txt")

    if len(inact_list) >= len(act_list):
        inact_list = inact_list[:len(act_list)]
    else:
        act_list = act_list[:int(len(inact_list)*1.5)]

    act_dict = dict()
    inact_dict = dict()
    for act_comp in act_list:
        act_dict[act_comp] = ""

    for inact_comp in inact_list:
        inact_dict[inact_comp] = ""
    # ecfp4_fl = open("/Users/trman/OneDrive/Projects/BioactivityDataAnalysis/trainingFiles/compound_ecfp4_vectors.tsv", "r")
    # lst_ecfp4_fl = ecfp4_fl.read().split("\n")
    # ecfp4_fl.close()
    ecfp4_dict = dict()
    count = 0

    for act_comp in act_dict.keys():
        act_smiles = ""
        try:
            act_smiles = compound_smiles_dict[act_comp]
        except:
            pass
        if act_smiles != "":
            try:

                m = Chem.MolFromSmiles(act_smiles)
                str_ecfp4 = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024).ToBitString()
                str_ecfp4 = [float(i) for i in str_ecfp4]
                ecfp4_dict[act_comp] = str_ecfp4

            except:
                pass


    for inact_comp in inact_dict.keys():
        inact_smiles = ""
        try:
            inact_smiles = compound_smiles_dict[inact_comp]
        except:
            pass
        if inact_smiles != "":
            try:

                m = Chem.MolFromSmiles(inact_smiles)
                str_ecfp4 = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024).ToBitString()
                str_ecfp4 = [float(i) for i in str_ecfp4]
                ecfp4_dict[inact_comp] = str_ecfp4
            except:
                pass
    # print(len(ecfp4_dict.keys()))
    """
    # using ready-to-use feature vectors
    with open("/Users/trman/OneDrive/Projects/BioactivityDataAnalysis/trainingFiles/compound_ecfp4_vectors.tsv") as f:
        for ecfp_line in f:
            count+= 1
            #if count%1000==0:
            #    print(count)
            ecfp_line = ecfp_line.split("\n")[0]
            comp_id, fp = ecfp_line.split("\t")
            try:
                act_dict[comp_id]
                fp = [float(i) for i in fp]
                ecfp4_dict[comp_id] = fp
            except:
                pass
            try:
                inact_dict[comp_id]
                fp = [float(i) for i in fp]
                ecfp4_dict[comp_id] = fp
            except:
                pass
    """

    # print("Number of active compounds :\t{}".format(len(act_list)))
    # print("Number of inactive compounds :\t{}".format(len(inact_list)))
    train_test_data = []
    for pos_comp in act_list:
        label = [1, 0]
        try:
            count += 1
            train_test_data.append([np.asarray(ecfp4_dict[pos_comp]), np.array(label), pos_comp])
        except:
            prob_count += 1
            pass

    for neg_comp in inact_list:
        label = [0, 1]
        try:
            count += 1
            train_test_data.append([np.asarray(ecfp4_dict[neg_comp]), np.array(label), neg_comp])
        except:
            prob_count += 1
            pass
    shuffle(train_test_data)
    return train_test_data

# def copyBestModelLOG(target):
#     best_result_fl = open("../resultFiles/ChEMBLBestModelResultsAll_v2.txt.txt", "r")
#     lst_best_result_fl = best_result_fl.read().split("\n")
#     best_result_fl.close()

#     log_fl_name = ""
#     for line in lst_best_result_fl:
#         line = line.split("\t")
#         print(line)
#         if line[2]==target:
#             log_fl_name = line[0]
#             break

#     print(log_fl_name)

#     log_path = ["rahmetCLUSTERLOGS", "tdoganYODALOGS"]
#     modelTypeLOG = ["ImageNetLOGS", "OtherLOGS"]


#     test_line = ""
#     for lgp in log_path:
#         for mdltype in modelTypeLOG:
#             if os.path.exists("../resultFiles/LOGS/{}/{}/{}".format(lgp, mdltype,log_fl_name)):
#                 subprocess.call(["cp","../resultFiles/LOGS/{}/{}/{}".format(lgp, mdltype,log_fl_name),"../resultFiles/LOGS/bestModelLOGS"])

# def getTestCompounds(target):

#     log_fl_name = getTopNModels(1)[0][target][0][2]

#     #print(log_fl_name)

#     test_line = ""
#     log_fl = open("../resultFiles/LOGS/bestModelLOGS/{}".format(log_fl_name),"r")
#     lst_log_fl = log_fl.read().split("\n")
#     log_fl.close()

#     for line in lst_log_fl:
#         if "ACT," in line:
#             test_line = line
#             break

#     test_comp_names = []

#     test_line = test_line.split("\t")
#     for item in test_line:
#         item = item.split(",")
#         if item[0].startswith("CHEMBL"):
#             test_comp_names.append(item[0])
#     #print(test_comp_names)
#     return test_comp_names


# def trainModelTarget(target, rotate=False):
#     print("1")
#     model = None
#     test_comps = ["CHEMBL63965"]
#     # print(test_comps)
#     print("2")
#     train = constructDataMatricesForATargetOtherClassifier(TEMP_IMG_OUTPUT_PATH, target, rotate)
#     train_comp_name = [i[2] for i in train]
#     print("3")
#     X = []
#     Y = []
#     for i in train:
#         if i[0].shape!=():
#             if i[2] not in test_comps:
#                 X.append(i[0].flatten())
#                 if i[1][0]==1:
#                     Y.append(1)
#                 else:
#                     Y.append(0)
#     X = np.array(X)


#     test_comp_name = []
#     test_y = []
#     test_x = []
#     for i in train:
#         if i[0].shape != ():
#             if i[2] in test_comps:
#                 test_x.append(i[0].flatten())
#                 test_comp_name.append(i[2])
#                 if i[1][0]==1:
#                     test_y.append(1)
#                 else:
#                     test_y.append(0)

#     #print(len(X), len(test_y))
#     clf = RandomForestClassifier(n_jobs=2)
#     #print(len(X), len(Y))
#     clf.fit(X, Y)
#     y_rf_test_pred = clf.predict(test_x)
#     print("Target:\t{}".format(target))
#     str_preds = ""
#     for ind in range(len(test_comp_name)):
#         str_preds += test_comp_name[ind]
#         if test_y[ind]==1 and y_rf_test_pred[ind]==1:
#             str_preds += ",TP,ACT,1"
#         elif test_y[ind]==1 and y_rf_test_pred[ind]==0:
#             str_preds += ",FN,ACT,0"
#         elif test_y[ind]==0 and y_rf_test_pred[ind]==0:
#             str_preds += ",TN,INACT,0"
#         else:
#             str_preds += ",FP,INACT,1"
#         str_preds += "\t"

#     print("RF Predictions:")
#     print(str_preds)

#     svc = svm.SVC()
#     svc.fit(X, Y)
#     y_svc_test_pred = svc.predict(test_x)

#     str_preds = ""

#     for ind in range(len(test_comp_name)):
#         str_preds += test_comp_name[ind]
#         if test_y[ind]==1 and y_svc_test_pred[ind]==1:
#             str_preds += ",TP,ACT,1"
#         elif test_y[ind]==1 and y_svc_test_pred[ind]==0:
#             str_preds += ",FN,ACT,0"
#         elif test_y[ind]==0 and y_svc_test_pred[ind]==0:
#             str_preds += ",TN,INACT,0"
#         else:
#             str_preds += ",FP,INACT,1"
#         str_preds += "\t"

#     print("SVM Predictions:")
#     print(str_preds)


#     lr = LogisticRegression()
#     lr.fit(X, Y)
#     y_lr_test_pred = lr.predict(test_x)

#     str_preds = ""

#     for ind in range(len(test_comp_name)):
#         str_preds += test_comp_name[ind]
#         if test_y[ind] == 1 and y_lr_test_pred[ind] == 1:
#             str_preds += ",TP,ACT,1"
#         elif test_y[ind] == 1 and y_lr_test_pred[ind] == 0:
#             str_preds += ",FN,ACT,0"
#         elif test_y[ind] == 0 and y_lr_test_pred[ind] == 0:
#             str_preds += ",TN,INACT,0"
#         else:
#             str_preds += ",FP,INACT,1"
#         str_preds += "\t"
#     print("LR Predictions:")
#     print(str_preds)

#     print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(target,accuracy_score(test_y, y_rf_test_pred), f1_score(test_y, y_rf_test_pred), matthews_corrcoef(test_y, y_rf_test_pred), accuracy_score(test_y, y_svc_test_pred), f1_score(test_y, y_svc_test_pred),
#                               matthews_corrcoef(test_y, y_svc_test_pred), accuracy_score(test_y, y_lr_test_pred), f1_score(test_y, y_lr_test_pred),
#                                           matthews_corrcoef(test_y, y_lr_test_pred)))

        

def trainModelECFP4Target(target):

    model = None
    test_comps = ["CHEMBL286"]

    train = constructECFP4FeaturesVectors(target)

    X = []
    Y = []
    train_comp_names = []
    for i in train:
        if i[0].shape!=():
            if i[2] not in test_comps:
                train_comp_names.append(i[2])
                X.append(i[0])
                if i[1][0]==1:
                    Y.append(1)
                else:
                    Y.append(0)
    X = X[:int(len(X)*0.64)]
    Y = Y[:int(len(Y)*0.64)]
    train_comp_names = train_comp_names[:int(len(train_comp_names)*0.64)]
    X = np.array(X)
    # print(train_comp_names)
    # print(test_comps)
    # print(set(train_comp_names)&set(test_comps))
    test_comp_name = []
    test_y = []
    test_x = []
    for i in train:
        if i[0].shape != ():
            if i[2] in test_comps:
                test_x.append(i[0])
                test_comp_name.append(i[2])
                if i[1][0]==1:
                    test_y.append(1)
                else:
                    test_y.append(0)


    #print(len(X), len(test_y))
    clf = RandomForestClassifier(n_jobs=2)
    clf.fit(X, Y)
    y_rf_test_pred = clf.predict(test_x)
    tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(test_y, y_rf_test_pred).ravel()
    # print(len(test_x))
    # print("Target:\t{}".format(target))
    str_preds = ""
    for ind in range(len(test_comp_name)):
        str_preds += test_comp_name[ind]
        if test_y[ind]==1 and y_rf_test_pred[ind]==1:
            str_preds += ",TP,ACT,1"
        elif test_y[ind]==1 and y_rf_test_pred[ind]==0:
            str_preds += ",FN,ACT,0"
        elif test_y[ind]==0 and y_rf_test_pred[ind]==0:
            str_preds += ",TN,INACT,0"
        else:
            str_preds += ",FP,INACT,1"
        str_preds += "\t"
    """
    print("RF Predictions:")
    print(str_preds)
    """
    svc = svm.SVC()
    svc.fit(X, Y)
    y_svc_test_pred = svc.predict(test_x)
    tn_svc, fp_svc, fn_svc, tp_svc = confusion_matrix(test_y, y_svc_test_pred).ravel()
    str_preds = ""

    for ind in range(len(test_comp_name)):
        str_preds += test_comp_name[ind]
        if test_y[ind]==1 and y_svc_test_pred[ind]==1:
            str_preds += ",TP,ACT,1"
        elif test_y[ind]==1 and y_svc_test_pred[ind]==0:
            str_preds += ",FN,ACT,0"
        elif test_y[ind]==0 and y_svc_test_pred[ind]==0:
            str_preds += ",TN,INACT,0"
        else:
            str_preds += ",FP,INACT,1"
        str_preds += "\t"
    """
    print("SVM Predictions:")
    print(str_preds)
    """

    lr = LogisticRegression()
    lr.fit(X, Y)
    y_lr_test_pred = lr.predict(test_x)
    tn_lr, fp_lr, fn_lr, tp_lr = confusion_matrix(test_y, y_lr_test_pred).ravel()

    str_preds = ""
    #print(test_comp_name)
    #print()
    for ind in range(len(test_comp_name)):
        str_preds += test_comp_name[ind]
        if test_y[ind] == 1 and y_lr_test_pred[ind] == 1:
            str_preds += ",TP,ACT,1"
        elif test_y[ind] == 1 and y_lr_test_pred[ind] == 0:
            str_preds += ",FN,ACT,0"
        elif test_y[ind] == 0 and y_lr_test_pred[ind] == 0:
            str_preds += ",TN,INACT,0"
        else:
            str_preds += ",FP,INACT,1"
        str_preds += "\t"
    """
    print("LR Predictions:")
    print(str_preds)
    """
    print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(target,
        accuracy_score(test_y, y_rf_test_pred), f1_score(test_y, y_rf_test_pred), matthews_corrcoef(test_y, y_rf_test_pred),
        tn_rf, fp_rf, fn_rf, tp_rf,
        accuracy_score(test_y, y_svc_test_pred), f1_score(test_y, y_svc_test_pred),matthews_corrcoef(test_y, y_svc_test_pred),
        tn_svc, fp_svc, fn_svc, tp_svc,
        accuracy_score(test_y, y_lr_test_pred), f1_score(test_y, y_lr_test_pred),matthews_corrcoef(test_y, y_lr_test_pred),
        tn_lr, fp_lr, fn_lr, tp_lr))

targets = ["CHEMBL286"]
count  = 0
for trgt in targets:
    count+=1
    eprint(count)
    trainModelECFP4Target(trgt)

ValueError: unknown locale: UTF-8